# 2020: Week 16

In [6]:
# Setup
import pandas as pd

In [7]:
# Load data
lewis = pd.read_excel(".\\inputs\\Can't Desktop Prep this-2.xlsx", "Lewisham Sales")
camden = pd.read_excel(".\\inputs\\Can't Desktop Prep this-2.xlsx", "Camden Sales")
staff = pd.read_excel(".\\inputs\\Can't Desktop Prep this-2.xlsx", "Staff days worked")

## Clean

In [8]:
#clean staff data
staff["Month"] = pd.to_datetime(staff["Month"], format="%d/%m/%Y")
staff = staff.melt(id_vars=["Month"],
          var_name = "Store",
          value_name ="Staff days worked")

staff.head()

,Month,Store,Staff days worked
0,2020-01-01,Lewisham,200
1,2020-02-01,Lewisham,190
2,2020-03-01,Lewisham,200
3,2020-04-01,Lewisham,60
4,2020-01-01,Camden,100


In [9]:
#create function to clean sales dataframes
def clean_df(df):
    df = df.melt(id_vars=["Category", "Scent"], 
            var_name="Date", 
            value_name="Value")
    df["Type"] = df["Date"].str.split(" ").str[0]
    df["Date"] = df["Date"].str.split(" ").str[1]
    df = (df.reset_index()
          .pivot_table('Value', ['Category', 'Scent', 'Date'], 'Type')
          .reset_index(drop=False))
    df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
    del df.columns.name
    return df

In [10]:
# Clean sales dataframes
lewis = clean_df(lewis)
lewis["Store"] = 'Lewisham'

camden = clean_df(camden)
camden["Store"] = 'Camden'

#join all data frames
df = pd.concat([lewis, camden])
df = pd.merge(df, staff, left_on = ["Date", "Store"], right_on=["Month", "Store"], how='left').drop("Month", axis=1)
df.head()

,Category,Scent,Date,Profit,Sales,Store,Staff days worked
0,Bar,Lemon,2020-01-01,-219,4222,Lewisham,200
1,Bar,Lemon,2020-02-01,881,4969,Lewisham,190
2,Bar,Lemon,2020-03-01,-658,1530,Lewisham,200
3,Bar,Lemon,2020-04-01,868,1388,Lewisham,60
4,Bar,Mint,2020-01-01,619,4118,Lewisham,200


## Export

In [14]:
df.to_csv(".\\outputs\\2020-16_data-preppin-output.csv", index=False)